In [63]:
import pandas as pd
import calendar
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Read in the data

In [43]:
click_df = pd.read_csv("clicks_fact.csv")
click_df

,Date,site_id,brand_id,clicks,signups,ftds,cpa_income,rev_income
0,1/1/2021,1,4,71.0,77,40,898,395
1,1/2/2021,1,2,43.0,45,46,1220,1098
2,1/3/2021,1,2,37.0,69,40,1217,188
3,1/4/2021,1,1,64.0,76,16,162,413
4,1/5/2021,1,2,113.0,66,58,1108,1171
...,...,...,...,...,...,...,...,...
1539,7/8/2021,8,1,55.0,48,56,1591,1394
1540,7/9/2021,8,1,96.0,64,62,533,1502
1541,7/10/2021,8,2,72.0,79,12,1821,1711
1542,7/11/2021,8,1,92.0,59,28,1242,643


In [44]:
site_df = pd.read_csv("company_map.csv")
site_df

,site_name,site_id
0,195bet,1
1,adriaticmarine.no,2
2,antek.se,3
3,arisitum.org,4
4,bingobrickan.se,5
5,casinoistockholm.se,6
6,grattage.com,7
7,natcasino.nu,8


In [45]:
brand_df = pd.read_csv("brand_map.csv")
brand_df

,brand_name,brand_id
0,LeoVegas,1
1,Bet365,2
2,Betfair,3
3,Bertil,4



# Task 1

Create a new table with aggregated data per:
1. day
2. site name
3. brand name

There are 1351 duplicated dates, which means the data is un-aggregated


In [46]:
click_df.loc[click_df.Date.duplicated(), :]

,Date,site_id,brand_id,clicks,signups,ftds,cpa_income,rev_income
193,1/1/2021,2,1,45.0,71,23,1143,351
194,1/2/2021,2,1,102.0,87,58,1478,1492
195,1/3/2021,2,1,56.0,35,33,1588,1692
196,1/4/2021,2,4,99.0,39,64,590,1668
197,1/5/2021,2,2,91.0,55,16,1049,1668
...,...,...,...,...,...,...,...,...
1539,7/8/2021,8,1,55.0,48,56,1591,1394
1540,7/9/2021,8,1,96.0,64,62,533,1502
1541,7/10/2021,8,2,72.0,79,12,1821,1711
1542,7/11/2021,8,1,92.0,59,28,1242,643



### Fill null values


In [47]:
click_df = click_df.fillna(0)


## Create a table that shows the daily Use and Financial statistics for each brand and site

This table creates aggregate columns at the grain, per day / per site / per brand id
The aggregate columns are:

1. clicks
2. signups
3. ftds
4. cpa_income
5. rev_income

(These will be renamed later on)

In [48]:
# per_day
daily_click_stats_df = click_df.groupby(['Date', "site_id", "brand_id"])["clicks", "signups", "ftds", 'cpa_income', 'rev_income'].apply(lambda x : x.astype(int).sum())
daily_click_stats_df

<ipython-input-48-bb9d4c671121>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  daily_click_stats_df = click_df.groupby(['Date', "site_id", "brand_id"])["clicks", "signups", "ftds", 'cpa_income', 'rev_income'].apply(lambda x : x.astype(int).sum())


clicks  signups  ftds  cpa_income  rev_income
Date     site_id brand_id                                               
1/1/2021 1       4             71       77    40         898         395
         2       1             45       71    23        1143         351
         3       1            100       42    39        1769        1057
         4       2             72       51    57        1395         641
         5       3              0       87    26        1876         964
...                           ...      ...   ...         ...         ...
7/9/2021 4       4             34       67    44        1926         400
         5       1             84       83    53        1144         116
         6       4             53       36    22        1413        1637
         7       1            106       61    23         888        1597
         8       1             96       64    62         533        1502

[1544 rows x 5 columns]


### Data Cleansing

Currently, the column 'Date' is an index, meaning that is has no associated data type. The Date column needs to be converted to the data type "datetime" so we can use in in upcoming transformations. 

1. Create a new index 
2. Set the Date column to be a datetime data type.
3. Check that the data type changed


In [49]:
daily_click_stats_df = daily_click_stats_df.reset_index()
daily_click_stats_df

,Date,site_id,brand_id,clicks,signups,ftds,cpa_income,rev_income
0,1/1/2021,1,4,71,77,40,898,395
1,1/1/2021,2,1,45,71,23,1143,351
2,1/1/2021,3,1,100,42,39,1769,1057
3,1/1/2021,4,2,72,51,57,1395,641
4,1/1/2021,5,3,0,87,26,1876,964
...,...,...,...,...,...,...,...,...
1539,7/9/2021,4,4,34,67,44,1926,400
1540,7/9/2021,5,1,84,83,53,1144,116
1541,7/9/2021,6,4,53,36,22,1413,1637
1542,7/9/2021,7,1,106,61,23,888,1597



####  This block of code shows the data type of Date is object


In [50]:
print(daily_click_stats_df.dtypes)

Date          object
site_id        int64
brand_id       int64
clicks         int64
signups        int64
ftds           int64
cpa_income     int64
rev_income     int64
dtype: object


In [51]:
daily_click_stats_df['Date'] = pd.to_datetime(daily_click_stats_df['Date'], format='%m/%d/%Y')
daily_click_stats_df

,Date,site_id,brand_id,clicks,signups,ftds,cpa_income,rev_income
0,2021-01-01,1,4,71,77,40,898,395
1,2021-01-01,2,1,45,71,23,1143,351
2,2021-01-01,3,1,100,42,39,1769,1057
3,2021-01-01,4,2,72,51,57,1395,641
4,2021-01-01,5,3,0,87,26,1876,964
...,...,...,...,...,...,...,...,...
1539,2021-07-09,4,4,34,67,44,1926,400
1540,2021-07-09,5,1,84,83,53,1144,116
1541,2021-07-09,6,4,53,36,22,1413,1637
1542,2021-07-09,7,1,106,61,23,888,1597



####  Now the date has been converted to the correct date type, and the data is clensed enough to move onto the next section


In [52]:
print(daily_click_stats_df.dtypes)


Date          datetime64[ns]
site_id                int64
brand_id               int64
clicks                 int64
signups                int64
ftds                   int64
cpa_income             int64
rev_income             int64
dtype: object


# Step 2: Insert the following columns
1. Date (month)
2. site_name
3. brand_name
4. total_clicks (this has already been done, rename)
5. total_signups (this has already been done, rename)
6. total_ftds (this has already been done, rename)
7. total_cpa_income (this has already been done, rename)
8. total_revenue

In [53]:
# 1. Parse out month into a seperate column
daily_click_stats_df['month'] = pd.DatetimeIndex(daily_click_stats_df['Date']).month
daily_click_stats_df['month'] = daily_click_stats_df['month'].apply(lambda x: calendar.month_abbr[x])
daily_click_stats_df2 = pd.merge(daily_click_stats_df, site_df, how="inner", on="site_id")
daily_click_stats_df3 = pd.merge(daily_click_stats_df2, brand_df, how="inner", on="brand_id")
daily_click_stats_df3["total_revenue"] = daily_click_stats_df3['cpa_income'] + daily_click_stats_df3['rev_income']
daily_click_stats_df3 = daily_click_stats_df3[["site_id", "brand_id", "Date", "month", "site_name", "brand_name", "clicks", "signups", "ftds", "cpa_income", "rev_income","total_revenue"]]
daily_click_stats_df3 = daily_click_stats_df3.rename(columns={'clicks': 'total_clicks', 'signups': 'total_signups', 'ftds':'total_ftds', 'cpa_income':'total_cpa_income', 'rev_income': 'total_rev_income'})

daily_click_stats_df3

,site_id,brand_id,Date,month,site_name,brand_name,total_clicks,total_signups,total_ftds,total_cpa_income,total_rev_income,total_revenue
0,1,4,2021-01-01,Jan,195bet,Bertil,71,77,40,898,395,1293
1,1,4,2021-01-13,Jan,195bet,Bertil,67,67,32,1340,1528,2868
2,1,4,2021-01-14,Jan,195bet,Bertil,37,83,17,447,300,747
3,1,4,2021-01-19,Jan,195bet,Bertil,87,84,67,734,525,1259
4,1,4,2021-01-20,Jan,195bet,Bertil,86,46,46,626,289,915
...,...,...,...,...,...,...,...,...,...,...,...,...
1539,8,2,2021-06-13,Jun,natcasino.nu,Bet365,44,60,18,1645,727,2372
1540,8,2,2021-06-14,Jun,natcasino.nu,Bet365,80,55,65,104,937,1041
1541,8,2,2021-06-07,Jun,natcasino.nu,Bet365,85,35,27,1393,1241,2634
1542,8,2,2021-07-10,Jul,natcasino.nu,Bet365,72,79,12,1821,1711,3532



# Step 3: Create a table showing the bottom 4 sites in terms of Total Revenue


Include 3 columns: 
1. Rank
2. Site Name
4. Total Revenue


I left the Euro symbol out, this can be added back within the API

In [56]:
bottom_four_revenue_df = daily_click_stats_df3[['month', 'site_name', 'total_revenue']]
# Group by the site name month and and aggregate yearly sum
bottom_four_revenue_df2 = bottom_four_revenue_df.groupby(['month', "site_name"])['total_revenue'].apply(lambda x : x.astype(int).sum())
# Get the 4 smallest total revenues
bottom_four_revenue_df3 = bottom_four_revenue_df.groupby(["site_name"])['total_revenue'].apply(lambda x : x.astype(int).sum()).nsmallest(4).reset_index()
# Sort by descending
bottom_four_revenue_df3 = bottom_four_revenue_df3.sort_values(by='total_revenue', ascending=False)


# Add the Rank Column
idx = 0
new_col = [5, 6, 7, 8]
bottom_four_revenue_df3.insert(loc=idx, column='Rank', value=new_col)

# Rename columns
bottom_four_revenue_df3 = bottom_four_revenue_df3.rename(columns={'site_name': 'Site Name', 'total_revenue': 'Total Revenue'})
bottom_four_revenue_df3


,Rank,Site Name,Total Revenue
3,5,casinoistockholm.se,388929
2,6,bingobrickan.se,387691
1,7,adriaticmarine.no,385964
0,8,grattage.com,379945



# Question 3: 
Assuming the table from part 1 has grown for a couple of years and contains more than 100 million rows, how would you approach adding an additional column to the aggregated table and populate it with data from a new source?  
The new source also has site_id and brand_id available  

# Answer
I have provided two solutions, each of the solutions differs by locking the table in production so no transactions can occur while we add the new column. This decision would need to be based on how much impact locking the table would have on the company as none of the users could make transactions during this time. Since RakeTech uses click and transaction data across multiple websites, I would lean towards solution #2, which has no downtime for the users. 


## Solution 1

1. Lock the table against concurrent changes in share mode, but allow reads.
2. Copy the original table. The COPY command is optimized for loading large numbers of rows; it is less flexible than INSERT, but incurs significantly less overhead for large data loads. It's fastest to drop indexes in this step.
3. Populate data
4. Add indexes with ALTER TABLE as final step

#### This solution requires that we lock the table until the new table can be populated. The original table must be locked in order to prevent transactions from taking place and impacting data validity. This could be an issue depending on how long it takes to create the table

## Solution 2

Use the Percona Toolkit library
**pt-online-schema-change** emulates the way that MySQL alters tables internally, but it works on a copy of the table you wish to alter. This means that the original table is not locked, and clients may continue to read and change data in it.

**pt-online-schema-change** works by creating an empty copy of the table to alter, modifying it as desired, and then copying rows from the original table into the new table. When the copy is complete, it moves away the original table and replaces it with the new one. By default, it also drops the original table.

##### Code Example

```python
pt-online-schema-change --alter "ADD COLUMN c1 INT" D=daily_clicks,t=invalid_user
print s
```
 


# Question 4

If wanting to split the first table not only on brand and site, but also on country and visitor device (desktop/mobile) after it’s been added to raw source data, how would you approach it to let us break down the data also on these parameters? Can you think of other interesting ways to break down the data in different ways? 
As in the previous task assuming the table has grown for a couple of years, and also that the first table is already aggregated from multiple sources where the mentioned data is now available.


# Answer : Countries with the highest spenders 


This table shows the most revenue for each sign-up. This table can tell us which countries spend the most, which is probably a good place to run ad-campaigns. Additionally, we can see who spent the most in which month. For instance, consumers in Spain may spend more in the winter months, while those in Switzerland are buys skiing, and spend very little. Since I used a random number generator, these results will vary.

This table analyzes the user's 
1. country 
2. number of euros per sign up
3. by month

I have also added some mapping tables for country and device. This allows this dataset to be explored even further in the future. We could look:
1. The day of the week people sign up/spend the most money
2. The % increase/decrease by a rolling monthly bases
3. The number of times a user clicked using a desktop or laptop
4. Who are the high rollers? How many sites has a user signed up for? How much did they spend?



In [57]:
# Copy the first table
daily_click_stats_df_v2 = daily_click_stats_df.copy(deep=False)
# Add randomly generated user_id for fun
daily_click_stats_df_v2['user_id'] =  np.random.randint(0,5000, size=len(daily_click_stats_df_v2))
# Add randomly generated device_id
daily_click_stats_df_v2['device_id'] =  np.random.randint(2, size=len(daily_click_stats_df_v2))
# Add randomly generated country_id
daily_click_stats_df_v2['country_id'] =  np.random.randint(11, size=len(daily_click_stats_df_v2))

daily_click_stats_df_v2

,Date,site_id,brand_id,clicks,signups,ftds,cpa_income,rev_income,month,user_id,device_id,country_id
0,2021-01-01,1,4,71,77,40,898,395,Jan,3344,1,8
1,2021-01-01,2,1,45,71,23,1143,351,Jan,2277,0,2
2,2021-01-01,3,1,100,42,39,1769,1057,Jan,4224,0,8
3,2021-01-01,4,2,72,51,57,1395,641,Jan,2673,0,4
4,2021-01-01,5,3,0,87,26,1876,964,Jan,1817,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...
1539,2021-07-09,4,4,34,67,44,1926,400,Jul,570,1,5
1540,2021-07-09,5,1,84,83,53,1144,116,Jul,3088,1,7
1541,2021-07-09,6,4,53,36,22,1413,1637,Jul,3950,1,1
1542,2021-07-09,7,1,106,61,23,888,1597,Jul,206,1,6



## Create the Device and Country mapping tables


In [58]:
device_df = pd.DataFrame({"device": ["laptop", "desktop"], "device_id": [0, 1]})
device_df

,device,device_id
0,laptop,0
1,desktop,1


In [59]:
country_df =  pd.DataFrame({"country": ["Malta", "Spain", "France", "Poland", "Switzerland", "Sweeden", "Portugal", "Italy", "Romania", "Norway"],
     "country_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

country_df


,country,country_id
0,Malta,1
1,Spain,2
2,France,3
3,Poland,4
4,Switzerland,5
5,Sweeden,6
6,Portugal,7
7,Italy,8
8,Romania,9
9,Norway,10



## Join the country and device mapping tables


In [61]:
daily_click_stats_df_v3 = pd.merge(daily_click_stats_df_v2, country_df, how="left", on="country_id")
daily_click_stats_df_v3 = pd.merge(daily_click_stats_df_v3, device_df, how="left", on="device_id")

daily_click_stats_df_v3

,Date,site_id,brand_id,clicks,signups,ftds,cpa_income,rev_income,month,user_id,device_id,country_id,country,device
0,2021-01-01,1,4,71,77,40,898,395,Jan,3344,1,8,Italy,desktop
1,2021-01-01,2,1,45,71,23,1143,351,Jan,2277,0,2,Spain,laptop
2,2021-01-01,3,1,100,42,39,1769,1057,Jan,4224,0,8,Italy,laptop
3,2021-01-01,4,2,72,51,57,1395,641,Jan,2673,0,4,Poland,laptop
4,2021-01-01,5,3,0,87,26,1876,964,Jan,1817,0,7,Portugal,laptop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539,2021-07-09,4,4,34,67,44,1926,400,Jul,570,1,5,Switzerland,desktop
1540,2021-07-09,5,1,84,83,53,1144,116,Jul,3088,1,7,Portugal,desktop
1541,2021-07-09,6,4,53,36,22,1413,1637,Jul,3950,1,1,Malta,desktop
1542,2021-07-09,7,1,106,61,23,888,1597,Jul,206,1,6,Sweeden,desktop


In [64]:
# Select columns
daily_click_stats_df_v4 = daily_click_stats_df_v3[["month", "country", "signups", "cpa_income", "rev_income"]]
# Group by month and country and sum the income and signups
country_stats_df = daily_click_stats_df_v4.groupby(['month', "country"])['signups', 'cpa_income', 'rev_income'].apply(lambda x : x.astype(int).sum())
# reset index
country_stats_df = country_stats_df.reset_index()
# create total income column
country_stats_df["total_revenue"] = country_stats_df['cpa_income'] + country_stats_df['rev_income']
# select columns
country_stats_df2 = country_stats_df[["month", "country", "signups", "total_revenue"]]
# Create revenue per signup column
country_stats_df2["revenue per signup"] = country_stats_df2["total_revenue"] / country_stats_df2["signups"]
# Display the 10 countries with the most spending per sign up
country_stats_df2 = country_stats_df2.nlargest(10, 'revenue per signup', keep='last')

country_stats_df2

<ipython-input-64-598057882e1c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_stats_df2["revenue per signup"] = country_stats_df2["total_revenue"] / country_stats_df2["signups"]


,month,country,signups,total_revenue,revenue per signup
33,Jul,Norway,209,9592,45.894737
39,Jul,Switzerland,787,33027,41.965693
31,Jul,Italy,651,26930,41.367127
40,Jun,France,956,39537,41.356695
34,Jul,Poland,325,13358,41.101538
52,Mar,Malta,1620,64724,39.953086
10,Feb,France,1236,48208,39.003236
64,May,Poland,1447,55713,38.502419
67,May,Spain,893,33681,37.716685
59,Mar,Switzerland,1369,51351,37.509861


# Thank you! If you have any questions please feel free to write me. 